### 판결문 데이터 형식 수정

In [1]:
import json
import pandas as pd

from sklearn.model_selection import train_test_split

# CSV 파일을 로드합니다
data = pd.read_csv('./data/judgment.csv', skipinitialspace=True)
data[data['CASE_ID']==70265]

,CASE_ID,CASENAME,CASENUMBER,DATE,COURTNAME,CASETYPE,JUDGETYPE,CASEURL,DECISION,SUBSTANCE,REFERENCE,CONTENT,WINSTATUS
70,70265,근로기준법위반,2005노818,2005-11-18,울산지방법원,형사,판결 : 상고,/DRF/lawService.do?OC=wkdtmf357&target=prec&ID...,[1] 근로기준법 제8조에 정한 ‘영리로’의 의미 및 영리성 유무...,[1] 근로기준법 제8조(중간착취의 배제)는 누구든지 법률에 의하...,"[1] 근로기준법 제8조, 제110조 / [2] ...",【피 고 인】 피고인【항 소 인】 피고인【검 사】 전미화【변 호 인】 변호사 ...,DEFENDANT


In [2]:
# Extract cols
proc_data = data[['CASE_ID', 'CONTENT', 'WINSTATUS']]

# Add parties
proc_data.loc[:, 'first_party'] = '검    사'
proc_data.loc[:, 'second_party'] = '피고인'

# PLAINTIFF (검사 승) -> 1
# DEFENDANT (피고 승) -> 0
# AMBIGUOUS -> 제거
proc_data.loc[:, 'WINSTATUS'] = proc_data['WINSTATUS'].apply(lambda x: 1 if x == 'PLAINTIFF' else (0 if x == 'DEFENDANT' else None))
proc_data = proc_data.dropna(subset=['WINSTATUS'])
proc_data['WINSTATUS'] = proc_data['WINSTATUS'].astype(int)

# Change col name and order
proc_data = proc_data.rename(columns={'CASE_ID': 'ID', 'CONTENT': 'facts', 'WINSTATUS': 'first_party_winner'})
proc_data = proc_data[['ID', 'first_party', 'second_party', 'facts', 'first_party_winner']]

proc_data.head()

/var/folders/9z/hh1by56x5kv12t6_7g4k0ngh0000gn/T/ipykernel_19062/1823100713.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_data.loc[:, 'first_party'] = '검    사'
/var/folders/9z/hh1by56x5kv12t6_7g4k0ngh0000gn/T/ipykernel_19062/1823100713.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_data.loc[:, 'second_party'] = '피고인'


,ID,first_party,second_party,facts,first_party_winner
3,64518,검 사,피고인,【피 고 인】 【항 소 인】 피고인【검 사】 이원규【변 호 인】 변호사 하창우...,0
4,64519,검 사,피고인,【피 고 인】 【항 소 인】 피고인【검 사】 박성재【변 호 인】 법무법인 세인...,1
5,64520,검 사,피고인,【피 고 인】 【항 소 인】 피고인【검 사】 김완규【원심판결】 서울남부지방법원...,1
6,64521,검 사,피고인,【피 고 인】 【항 소 인】 피고인【검 사】 임무영【변 호 인】 변호사 한종원...,1
7,64522,검 사,피고인,【피 고 인】 【항 소 인】 쌍방【검 사】 임무영【변 호 인】 변호사 백성길(...,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6083 entries, 0 to 6082
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CASE_ID     6083 non-null   int64 
 1   CASENAME    6083 non-null   object
 2   CASENUMBER  6083 non-null   object
 3   DATE        6083 non-null   object
 4   COURTNAME   6083 non-null   object
 5   CASETYPE    6083 non-null   object
 6   JUDGETYPE   6083 non-null   object
 7   CASEURL     6083 non-null   object
 8   DECISION    2471 non-null   object
 9   SUBSTANCE   2434 non-null   object
 10  REFERENCE   2472 non-null   object
 11  CONTENT     6083 non-null   object
 12  WINSTATUS   6083 non-null   object
dtypes: int64(1), object(12)
memory usage: 617.9+ KB


In [4]:
proc_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5174 entries, 3 to 6082
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  5174 non-null   int64 
 1   first_party         5174 non-null   object
 2   second_party        5174 non-null   object
 3   facts               5174 non-null   object
 4   first_party_winner  5174 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 242.5+ KB


### Train, Test split

In [5]:
# 데이터를 80:20 비율로 나눕니다
train_data, test_data = train_test_split(proc_data, test_size=0.2, random_state=42)

train_data['ID'] = train_data['ID'].apply(lambda x: 'TRAIN_' + str(x))
test_data['ID'] = test_data['ID'].apply(lambda x: 'TEST_' + str(x))

# 결과를 각각 CSV 파일로 저장합니다
train_data.to_csv('./data/train.csv', index=False)
test_data.to_csv('./data/test.csv', index=False)

print("데이터가 성공적으로 80:20 비율로 나누어졌습니다.")

데이터가 성공적으로 80:20 비율로 나누어졌습니다.


In [6]:
def convert_csv2xlsx(domain='train'):
    data = []
    df = pd.read_csv(f"./data/{domain}.csv")

    for i in range(0, len(df)):
        data_id = df['ID'][i]
        users = [df['first_party'][i], df['second_party'][i]]
        text = df['facts'][i]

        for user_i in range(len(users)):
            if 'Title: \t ' in users[user_i]:
                users[user_i] = users[user_i].split('Title: \t ')[1]

        if domain == 'train':
            if int(df['first_party_winner'][i]) == 1: gt = 'Victory'
            else: gt = 'Defeat'
        else:
            gt = ''
        
        data.append(
            {
                'test_id': data_id,
                'The first party': users[0],
                'The second party': users[1], 
                'facts': text,
                'output': gt
            }
        )
    
    json.dump(data, open(f'./data/{domain}.json', 'w', encoding='utf-8'), indent='\t')

### Train, Validation Split

In [7]:
convert_csv2xlsx('train')
convert_csv2xlsx('test')

with open('./data/train.json', "r", encoding='utf-8') as file:
    data = json.load(file)

length = len(data) // 3
A_paths = data[:length]
B_paths = data[length:2*length]
C_paths = data[2*length:]

train_set, test_set = A_paths+C_paths, B_paths

with open(f'./data/trainval.json', 'w', encoding='utf-8') as file:
    json.dump({'train': train_set, 'validation': test_set}, file, indent='\t', ensure_ascii=False)